# Hands-on 1: FlowMonitor, campanhas e plotagem de gráficos

Este é um hands-on intermediário que deve ser feito obrigatoriamente antes das Campanhas 1, 2 e 3. Ele está dividido em 3 partes: 

**1) A inserção do módulo FlowMonitor no código de interesse;**

**2) A criação de campanhas de simulação e;**

**3) A plotagem de gráficos.**


## Requisitos:

1) Noção do ns3.29;

2) Noção do Matlab;

3) O script utilizado neste hands-on será o **rate-adaptation-distance.cc** do ns-3.

## Parte 1: Inserção do módulo FlowMonitor

O [FlowMonitor](https://www.nsnam.org/docs/release/3.29/models/html/flow-monitor.html) é um módulo do simulador de redes ns-3, utilizado na captura de pacotes para obter diversas métricas de rede. O objetivo do módulo é providenciar um sistema fléxivel para medir o desempenho de protocolos de rede. O módulo usa objetos chamados de *probes*, instalados nos nós, para rastrear os pacotes transmitidos entre os nós, e fará a medição de um certo número de parâmetros. Os pacotes são dividios de acordo com o fluxo em que eles pertencem, onde cada fluxo é definido de acordo com as características dos *probes* (por exemplo, para a camada IP, um fluxo é definido, por exemplo, como: pacotes com as mesmas {protocolos, portas, endereço IP} tuplas.

O uso do FlowMonitor padrão é bastante simples, sendo limitado à um pequeno número de linhas:

Fonte: [Documentação](https://www.nsnam.org/docs/release/3.29/models/html/flow-monitor.html)

In [ ]:
#include "ns3/flow-monitor-module.h"
#include "ns3/flow-monitor-helper.h"
#include "ns3/flow-probe.h"


...

// Flow monitor
Ptr<FlowMonitor> flowMonitor;
FlowMonitorHelper flowHelper;
flowMonitor = flowHelper.InstallAll();

Simulator::Stop (Seconds(stop_time));
Simulator::Run ();

flowMonitor->SerializeToXmlFile("NameOfFile.xml", true, true); 

Ao inserir as linhas de código acima em seu código, você perceberá que um arquivo ".xml" será gerado com as as informações dos fluxos. Contudo, um arquivo do tipo ".xml" não é o melhor formato nem possui a melhor forma de visualização para adquirir as principais métricas de rede. Para isso, foi desenvolvido algumas linhas de código em c++, que utiliza os objetos criados pelo FlowMonitor para captar as métricas calculados pelo FlowMonitor e salvá-las em um arquivo ".txt".

In [ ]:
//////////////////// Flow Monitor /////////////////////////
AsciiTraceHelper asciiTraceHelper;
Ptr<FlowMonitor> flowMonitor;
FlowMonitorHelper flowHelper;
flowMonitor = flowHelper.InstallAll();

//////////////////////////////////////////////////////////
Simulator::Stop (Seconds (simuTime));
Simulator::Run ();

////////Flow Monitor data///////////////////////////////
static bool verbose = true;
uint32_t m_bytesTotal=0;
std::string dl_results,ul_results;
dl_results = "DL_Results_Sim_PropModel_.txt";
ul_results = "UL_Results_Sim_PropModel_.txt";


Ptr<OutputStreamWrapper> DLstreamMetricsInit = asciiTraceHelper.CreateFileStream((dl_results));
*DLstreamMetricsInit->GetStream()
            << "Flow_ID, Lost_Packets, Tx_Packets, Tx_Bytes, TxOffered(Mbps),  Rx_Packets, Rx_Bytes, T_put(Mbps), Mean_Delay_Rx_Packets, Mean_Jitter, Packet_Loss_Ratio"
            << std::endl;

Ptr<OutputStreamWrapper> ULstreamMetricsInit = asciiTraceHelper.CreateFileStream((ul_results));
*ULstreamMetricsInit->GetStream()
            << "Flow_ID, Lost_Packets, Tx_Packets, Tx_Bytes, TxOffered(Mbps),  Rx_Packets, Rx_Bytes, T_put(Mbps), Mean_Delay_Rx_Packets, Mean_Jitter, Packet_Loss_Ratio"
            << std::endl;

double statDurationTX = 0;
double statDurationRX = 0;
Ptr<Ipv4FlowClassifier> classifier = DynamicCast<Ipv4FlowClassifier>(flowHelper.GetClassifier());

std::map<FlowId, FlowMonitor::FlowStats> stats = flowMonitor->GetFlowStats();
for (std::map<FlowId, FlowMonitor::FlowStats>::const_iterator iter =
    stats.begin(); iter != stats.end(); ++iter)
  {
    // some metrics calculation
    statDurationRX = iter->second.timeLastRxPacket.GetSeconds()
                  - iter->second.timeFirstTxPacket.GetSeconds();
    statDurationTX = iter->second.timeLastTxPacket.GetSeconds()
                  - iter->second.timeFirstTxPacket.GetSeconds();

    double meanDelay, meanJitter, packetLossRatio, txTput, rxTput;
    if (iter->second.rxPackets > 0)
      {
        meanDelay = (iter->second.delaySum.GetSeconds()
            / iter->second.rxPackets);
      }
    else // this value is set to zero because the STA is not receiving any packet
      {
        meanDelay = 0;
      }
    //
    if (iter->second.rxPackets > 1)
      {
        meanJitter = (iter->second.jitterSum.GetSeconds()
            / (iter->second.rxPackets - 1));
      }
    else // this value is set to zero because the STA is not receiving any packet
      {
        meanJitter = 0;
      }
    //
    if (statDurationTX > 0)
      {
        txTput = iter->second.txBytes * 8.0 / statDurationTX / 1000 / 1000;
      }
    else
      {
        txTput = 0;
      }
    //
    if (statDurationRX > 0)
      {
        rxTput = iter->second.rxBytes * 8.0 / statDurationRX / 1000 / 1000;
      }
    else
      {
        rxTput = 0;
      }
    //
    if ((iter->second.lostPackets > 0) & (iter->second.rxPackets > 0))
      {
        packetLossRatio = (double) (iter->second.lostPackets
            / (double) (iter->second.rxPackets + iter->second.lostPackets));
      }
    else
      {
        packetLossRatio = 0;
      }
    //
    Ipv4FlowClassifier::FiveTuple t = classifier->FindFlow(iter->first);
    //
    if (verbose)
           {
             // Print information only if enabled
             //
             //          std::cout << "Flow ID: " << iter->first << ", Source Port: "
             //              << t.sourcePort << ", Destination Port: " << t.destinationPort
             //              << " (" << t.sourceAddress << " -> " << t.destinationAddress
             //              << ")" << std::endl;
             //
             NS_LOG_UNCOND( "Flow ID: " << iter->first << ", Source Port: "
                 << t.sourcePort << ", Destination Port: " << t.destinationPort
                 << " (" << t.sourceAddress << " -> " << t.destinationAddress
                 << ")");
             //
             NS_LOG_UNCOND( "Lost Packets = " << iter->second.lostPackets);
             //
             NS_LOG_UNCOND( "Tx Packets = " << iter->second.txPackets);
             //
             NS_LOG_UNCOND( "Tx Bytes = " << iter->second.txBytes);
             //
             NS_LOG_UNCOND( "TxOffered = " << txTput << " Mbps");
             //std::cout << "TxOffered = " << txTput << " Mbps" << std::endl;
             //
             NS_LOG_UNCOND( "Rx Packets = " << iter->second.rxPackets);
             //
             NS_LOG_UNCOND( "Rx Bytes = " << iter->second.rxBytes);
             //
             NS_LOG_UNCOND( "T-put = " << rxTput << " Mbps");
             //std::cout << "T-put = " << rxTput << " Mbps" << std::endl;
             //
             NS_LOG_UNCOND( "Mean Delay Rx Packets = " << meanDelay << " s");
             //std::cout << "Mean Delay Rx Packets = " << meanDelay << " s"
             //    << std::endl;
             //
             NS_LOG_UNCOND( "Mean jitter = " << meanJitter << " s");
             //std::cout << "Mean jitter = " << meanJitter << " s" << std::endl;
             //
             NS_LOG_UNCOND( "Packet loss ratio = " << packetLossRatio);
             //std::cout << "Packet loss ratio = " << packetLossRatio << std::endl;
             //
           }
         //
         Ptr<OutputStreamWrapper> streamMetricsInit = NULL;
         // Get file pointer for DL, if DL flow (using port and IP address to assure correct result)
         std::cout << "t destination port " << t.destinationPort  <<std::endl;
         //std::cout << "source address " << interfaces.GetAddress(0)  <<std::endl;
         std::cout << "source address " << t.sourceAddress  <<std::endl;
         std::cout << "t destination port " << t.destinationPort  <<std::endl;
         std::cout << "sink address " << sinkAddress  <<std::endl;
         std::cout << "destination address " << t.destinationAddress  <<std::endl;
         if ((t.destinationPort == port)
             & (t.sourceAddress == i.GetAddress (1)))
           {
             streamMetricsInit = DLstreamMetricsInit;
           }
         // Get file pointer for UL, if UL flow (using port and IP address to assure correct result))
         else if ((t.destinationPort == port)
             & (t.destinationAddress == sinkAddress))
           {
             streamMetricsInit = ULstreamMetricsInit;
           }
         //
         if (streamMetricsInit)
           {
             *streamMetricsInit->GetStream() << (iter->first ) << ", "
                 << (iter->second.lostPackets) << ", "
                 //
                 << (iter->second.txPackets) << ", "
                 //
                 << (iter->second.txBytes) << ", "
                 //
                 << txTput << ", "
                 //
                 << (iter->second.rxPackets) << ", "
                 //
                 << (iter->second.rxBytes) << ", "
                 //
                 << rxTput << ", "
                 //
                 << meanDelay << ", "
                 //
                 << meanJitter << ", "
                 //
                 << packetLossRatio 
                 //
                 << std::endl;
           }
         else
           {
             //TODO: chance for an ASSERT
             if ( true )
               {
                 std::cout << "Some problem to save metrics" << std::endl;
                 std::cout << "Flow ID: " << iter->first << ", Source Port: "
                     << t.sourcePort << ", Destination Port: " << t.destinationPort
                     << " (" << t.sourceAddress << " -> " << t.destinationAddress << ")" << std::endl;
                 std::cout << "AP Address: " << i.GetAddress(0) << std::endl;
                 std::cout << "DLport: " << port << std::endl;
                 std::cout << "ULport: " << port << std::endl;
               }
           }
         m_bytesTotal =+ iter->second.rxPackets;
  }

flowMonitor->SerializeToXmlFile("NameOfFile.xml", true, true);

/////////////
Simulator::Destroy ();


Algumas observações importantes sobre o código:

1) Ele deve estar inserido entre as linhas "Simulator::Run ();" e "Simulator::Destroy ();"

2) O nome do .txt a ser salvo no final da simulação deve ser compatível com seu problema, e são regidas pelas variáveis "dl_results" e "ul_results" para downlink e uplink, respectivamente. 

3) A variável iterador criada (*iter*) dentro do laço "for" é o objeto *probe* utilizado pelo FlowMonitor. Ele irá iterar por cada um dos nós da simulação, captando os pacotes e calculando e salvando as métricas em outras variáveis locais.

4) Se a variável *verbose* estiver habilitada (isto é, sendo *True*), uma série de logs é habilitado, e para cada simulação os logs são mostrados na tela do terminal.

5) O restante do código é responsável por salvar as métricas de rede no arquivo .txt.

6) IMPORTANTE: Esse trecho de código foi adaptado para um problema específico. Certamente será necessário adaptá-lo para o seu.

## Desafio 1: Adicionar o FlowMonitor.

1) Utilizando o script *rate-adaptation-distance.cc*, adicione o código acima, faça as mudanças necessárias, compile e rode.

2) Mude as variáveis para que a distância entre os nós seja fixa durante toda a simulação. O objetivo de fazer isso é diminuir o tempo de simulação.

Ao final da simulação, 3 arquivos devem ter sido gerados:

1) NameOfFile.xml: arquivo gerado pelo FlowMonitor, sem uso para nosso trabalho;

2) "DL_Results_Sim_PropModel_.txt", e;

3) "UL_Results_Sim_PropModel_.txt".

Para consulta, **os 3 arquivos gerados, bem como o script de simulação utilizado,** estão disponíveis em [aqui](CODE/).

## Parte 2: Criação das Campanhas de Simulação

## Introdução

Um dos pontos-chaves na análise de um sistema, independente da área de estudo, é a reprodução de resultados e investigação dos comportamentos. Na área de telecomunicação, tem-se a simulação como forma de rápida, barata e confiável de fazer essa investigação. É comum que *Open Source softwares* tenham todas essas três características, além, claro, da complexidade, que é inerente a cada ferramenta. Porém, na natureza há ainda uma variável chamada **aleatoriedade** que está inserida em praticamente todos as questões da natureza. Ela é, por exemplo, responsável por, ora seu aparelho móvel ter taxas de **throughputs** diferentes, ora não, embora não haja mudança aparente no sistema. Então, como é que se modela algo que em condições parecidas, entrega resultados diferentes? A resposta está na repetição do experimento um número suficiente de vezes para que o resultado entregue tenha uma certa precisão e confiabilidade. Idealmente, para se ter uma medida confiável do cenário analisado, as repetições deveriam tender ao infinito, contudo infinito não é praticável e, na maioria dos casos não é necessário: cada problema tem seu número mínimo de repetições. Nesse hands-on, cada repetição de cenário é chamado de **Job** e uma série de *jobs* formam uma campanha de simulação. 


As campanhas de simulação são criadas utilizando o software **Matlab**, e os scripts e arquivos utilizados nas campanhas estão todos disponíveis [aqui](CODE/) e são:

**1) createWifiSimCampaign.m: principal script para criação das campanhas.**

- Nessa função, há a variável **nRepetitions** que define a quantidade de Jobs.

**2) paramBaseWIFI.sim: Arquivo de texto com todas as variavéis globais e locais usadas na simulação, se nenhuma variável global for utilizada, o arquivo pode ser vazio - é utilizado para inicializar as variáveis.**

**3) WIFI_RateAlg_Load_CampaignSet.m: script em que todas as variáveis (e seus respectivos valores) escolhidas para análise devem ser colocadas de acordo com uma hierarquia.**
 
 - Todas as variáveis escolhidas para análise devem ser declaradas como do tipo **global** (no script em c++), além disso deve ser possível alterá-las via linha de comando;
 - Necessário adicionar ao seu script o módulo [ConfigStore](https://www.nsnam.org/docs/release/3.29/doxygen/classns3_1_1_config_store.html);
 - A documentação sobre GlobalVariables: [aqui](https://www.nsnam.org/docs/release/3.29/doxygen/classns3_1_1_global_value.html).

## Passos para adaptação do código


### Passo 1: Inserindo o módulo ConfigStore

Devido ao modo como as campanhas são feitas, é preciso utilizar o módulo *ConfigStore* para que a leitura pelo script dos parâmetros de cada *Job* seja feita por um arquivo de texto do tipo ".sim". Adicione as seguintes linhas de código ao seu código, por exemplo:

In [ ]:
#include <ns3/config-store-module.h>

...
CommandLine cmd;
cmd.AddValue("verbose", "Tell echo applications to log if true", verbose);
cmd.Parse (argc, argv);

ConfigStore inputConfig;
inputConfig.ConfigureDefaults ();
inputConfig.ConfigureAttributes ();

### Passo 2: Variáveis Globais

Todas as variáveis que serão objeto de análise devem ser declaradas como globais. Então, os códigos devem obedecer a seguinte estrutura no que se refere às **variávies globais e ConfigStore**:

In [ ]:
#include <ns3/config-store-module.h>
#include "ns3/core-module.h"

...


static ns3::GlobalValue g_myGlobal ("myGlobal",
                                     "My global value for ...",
                             ns3::UintegerValue (12),
                             ns3::MakeUintegerChecker<uint32_t> ());
    
int main (int argc, char *argv[])
{

CommandLine cmd;
cmd.Parse (argc, argv);

ConfigStore inputConfig;
inputConfig.ConfigureDefaults ();
inputConfig.ConfigureAttributes ();
    
BooleanValue booleanValue; //Declaring variables types

GlobalValue::GetValueByName ("myGlobal", booleanValue);
bool myGlobal = booleanValue.Get ();

inputConfig.ConfigureAttributes ();
    
...       
}

## Criação das campanhas


### Passo 1: Copiar arquivos
- Copiar todos os arquivos *.m em anexo para a pasta "/ns-allinone-3.29/ns-3.29/". E os arquivos *.cc para a pasta "/ns-allinone-3.29/ns-3.29/scratch".

### Passo 2:  Arquivo .m com a variação dos parâmetros

- Para criar uma campanha de simulação precisamos primeiramente criar um arquivo **.m** com a variação de parâmetros que desejamos. Esse arquivo é o *WIFI_RateAlg_Load_CampaignSet.m*. Verifique seu problema e faças as alterações necessárias.

### Passo 3: Script createXXXSimCampaign.m

- Usar o script **createWIFISimCampaign.m** para criar todas as pastas e arquivos necessários para rodar a campanha de simulação. Ele criará uma pasta chamada **WIFI_RateAlg_Load_CampaignSet** com todas as simulações especificadas no arquivo **WIFI_RateAlg_Load_CampaignSet.m**. Seu uso é como mostrado a seguir.

No Matlab, digite:

 - createWIFISimCampaign('/Users/gppcom/ns3/ns-allinone-3.29/ns-3.29/paramBaseWIFI.sim', '/Users/gppcom/ns3/ns-allinone-3.29/ns-3.29/WIFI_RateAlg_Load_CampaignSet.m', 'rate-adaptation-distance')

Necessário tomar cuidado com o caminho que está o código. Assim, adeque o comando acima de acordo com o caminho que está instalado o ns-3 no computador que você vai rodar. 

### Passo 4: Rodando as simulações

- A função **createWIFISimCampaign** também cria também um arquivo chamado WIFI_RateAlg_Load_CampaignSet.sh. Ele contém as chamadas de todas as simulações configuradas em WIFI_RateAlg_Load_CampaignSet.m. Para rodar as simulações, execute os seguintes comandos no terminal:

    chmod +x WIFI_RateAlg_Load_CampaignSet.sh

    ./WIFI_RateAlg_Load_CampaignSet.sh

Caso encontre algum erro, veja o que a mensagem de erro informa e tente interpretar. Alguns possíveis problemas que podem acontecer dizem respeito às permissões de acesso aos diretórios, pois podem ter sido criados usando o **sudo** (se seu Matlab tiver essa permissão, certamente esse problema ocorrerá). Esse problema pode ser resolvido ou retirando as permissões do diretório em questão ou criando outro com os mesmos arquivos e em seguida, deletando o diretório **sudo**, renomeando o criado para o nome correto. Se tudo ocorrer bem, a simulação deve começar:

![fig1](FIGS/fig1.png)

## Parte 3: Plot dos Gráficos

Os **principais** parâmetros utilizados na simulação feita foram:

- **std::string standard** = "802.11a";

- **uint32_t chWidth** = 20; 

- **int steps** = 1;

- **int stepsSize** = 1;

- **int stepsTime** = 1.

Para mais informações sobre os parâmetros, reveja o **Hands-on 01: Campanha 0**. E as variáveis foco dessa análise, são:

- **staManager:** MinstrelWifiManager;

- **apManager:** MinstrelWifiManager;

- **shortGuardInterval:** true (embora não haja diferença visto que o GuardInterval é fixo no 802.11a);

- **ap1_x:** {0,5,10} m (indica a posição em X do AP).


Importante notar que não é o objetivo deste hands-on fazer uma análise de desempenho completa sobre o tema, os gráficos mostrados a seguir devem ser utilizados como exemplo para ajudar nos plots da sua campanha.

Os gráficos são gerados utilizando o software **Matlab**, e os scripts estão todos disponíveis [aqui](CODE/) e são:

**1) plotWifiSimCampaign.m: principal script para geração dos gráficos.**


- Nessa função, há a variável **nRepetitions** que define a quantidade de Jobs.
- Uso: plotWifiSimCampaign('/Users/gppcom/ns3/ns-allinone-3.29/ns-3.29/Wifi_RateAlg_Load_CampaignSet.m')

**2) readMetrics.m: Utilizado pelo plotWifiSimCampaign para ler os principais parâmetros das variáveis.**

**3) calculateCI.m: calcula o intervalo de confiança.**
 

Para essa campanha-exemplo desenvolvida nesta parte deste hands-on, os parâmetros foram relaxadas para que a simulação fosse rapidamente reproduzida. O número de Jobs = 10 e para **ap1_x** apenas três valores foram utilizados. E dois gráficos foram gerados: 


![fig2](FIGS/fig2_plr-3.png)

![fig3](FIGS/fig3_tput.png)


Algumas conclusões sobre os gráficos: 1) ambos os gráficos apresentam grande intervalo de confiança grande, para que os resultados sejam mais confiáveis, é necessário que mais Jobs sejam feitos; 2) a taxa de perda de pacote se mantém dentro de uma faixa de média 93% para as distâncias analisadas, espera-se que esse valor aumente. Note que esse valor é bastante alto, porém veja que a **TxOffered = 407.925 Mbps** (reveja a figura do output do terminal), valor fixo para todas as simulações; 3) enquanto que a **vazão útil** (Tput) está numa faixa em que sua média é em torno de **27 Mbps**, corroborando o gráfico da **PLR** (*Packet Loss Ratio*). 

Importante destacar que os scripts disponibilizados foram adaptados para esse problema, para campanhas seguintes será necessário fazer as devidas modificações no script de plotagem para que seus gráficos sejam gerados sem maiores problemas.